In [ ]:
import numpy as np
import numpy_financial as npf
import pandas as pd
import plotly.express as px
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import collections

In [ ]:
# Parametros
precio_casa = 465000
ahorros = 125000
prestamo = 372000
itp = 0.06 # Impuesto transmisiones patrimoniales (Madrid)
notario = 1600
registro = 800
gestoria = 350
tasacion = 875
anios = 30
tin = 0.0129 # Tasa interes nominal (es anual no mensual)

In [ ]:
# # Calculos derivados
# def calc_prestamo_inicial(precio_casa, ahorros, default_inicial = None):
#     if not default_inicial:
#         prestamo = precio_casa - (ahorros - itp*precio_casa - notario - registro - gestoria - tasacion)
#     else:
#         prestamo = default_inicial
#     return prestamo

# def calc_serie_cuota_mensual(tin, anios, precio_casa, ahorros, as_array = True, default_inicial = None):
#     i = tin / 12
#     r = 1/(1+i)
#     meses = 12 * anios
#     prestamo = calc_prestamo_inicial(precio_casa, ahorros,default_inicial)
#     cuota = (1-r)*prestamo / (r - r**(meses+1)) # No uso aproximacion por exponencial
#     if as_array:
#         return cuota * np.ones(meses)
#     else:
#         return cuota

# def calc_series_mensuales_deudas_intereses_principal(tin, anios, prestamo,cuota_mensual,extra_amortizaciones=0):
#     i = tin/12
#     meses = 12 * anios
#     # Initialization
#     deuda_mensual_prepago = np.zeros(meses)
#     deuda_mensual_postpago = np.zeros(meses)
#     interes_mensual = np.zeros(meses)
#     principal_mensual = np.zeros(meses)
#     # Fisrt values
#     interes_mensual[0] = i * prestamo
#     deuda_mensual_prepago[0] = prestamo + interes_mensual[0] # El primer mes es el total mas el interes
#     deuda_mensual_postpago[0] = deuda_mensual_prepago[0] - cuota_mensual[0]
#     principal_mensual[0] = cuota_mensual[0] - interes_mensual[0]
    
#     for mes in range(1,meses):
#         interes_mensual[mes] = deuda_mensual_prepago[mes-1]*i
#         deuda_mensual_prepago[mes] = (deuda_mensual_prepago[mes-1] - cuota_mensual[mes-1]) + interes_mensual[mes]
#         deuda_mensual_postpago[mes] = deuda_mensual_prepago[mes] - cuota_mensual[mes]
#         principal_mensual[mes] = cuota_mensual[mes] - interes_mensual[mes]
        
#     return pd.DataFrame(data={"interes_mensual":interes_mensual, "deuda_mensual_prepago":deuda_mensual_prepago,"deuda_mensual_postpago":deuda_mensual_postpago, "principal_mensual":principal_mensual}, index=np.arange(1,meses+1))


In [ ]:
def calc_serie_cuota_mensual(tin, n_meses, prestamo, as_array = False):
    i = tin / 12
    r = 1/(1+i)
    cuota = (1-r)*prestamo / (r - r**(n_meses+1)) # = i * prestamo / (1 - (1+i)^(-n_meses))
    if as_array:
        return cuota * np.ones(n_meses)
    else:
        return cuota
    
def crear_serie_amortizaciones(amortizaciones_dic, total_anios):
    amortizaciones_anticipadas = np.zeros(total_anios*12)
    for anio,valor_amortizado in amortizaciones_dic.items():
        amortizaciones_anticipadas[12*anio-1] = valor_amortizado
            
    return amortizaciones_anticipadas

def crear_serie_tins(euribor_inicial, diferencial, variaciones_euribor_dic, total_anios):
    tin_mensuales = np.zeros(total_anios*12+1) # Tambien incluyo una posicion 0 inicial que solo capital vivo tiene
    tin_mensuales[1:13] = euribor_inicial + diferencial # Asumo scalar el diferencial por ahora, y solo cambia a final de anio
    for anio in range(1,total_anios):
        tin_mensuales[12*anio+1:12*(anio+1)+1] = tin_mensuales[12*(anio-1)+1:12*(anio)+1]
        if anio in variaciones_euribor_dic.keys():
            tin_mensuales[12*anio+1:12*(anio+1)+1] += variaciones_euribor_dic[anio]
            
    return tin_mensuales

def calc_series_mensuales_deudas_intereses_principal(tin, anios, prestamo,a_anticipadas=None):
    n_meses = 12 * anios
    
    # Initialization
    if not isinstance(tin,collections.Sequence):
        tin_mensual = np.ones(n_meses+1)*tin
    else:
        tin_mensual = tin
    capital_vivo = np.zeros(n_meses+1)
    capital_amortizado = np.zeros(n_meses+1)
    intereses = np.zeros(n_meses+1)
    amortizacion = np.zeros(n_meses+1)
    mensualidades=np.zeros(n_meses+1)
    mensualidades_con_amort=np.zeros(n_meses+1) 
    if a_anticipadas is None: # Si no hay amortizaciones anticipadas las inicializo todas a cero
        a_anticipadas = np.zeros(n_meses) # Luego se prependea un elemento siempre, por eso es n_meses long en lugar de n_meses+1
    
    # Mes 0 - Valores que no sean 0s
    capital_vivo[0] = prestamo
    a_anticipadas = np.insert(a_anticipadas, 0,0) # Prepend un 0 en la posicion 0
    
    for mes in range(1,n_meses+1):
        mensualidades[mes] = calc_serie_cuota_mensual(tin=tin_mensual[mes],n_meses=n_meses-mes+1,prestamo=capital_vivo[mes-1]) 
        mensualidades_con_amort[mes]=mensualidades[mes] + a_anticipadas[mes]
        intereses[mes] = capital_vivo[mes-1]*tin_mensual[mes]/12 # Pagas el interes sobre lo que falta por pagar (capital vivo)
        amortizacion[mes] = mensualidades_con_amort[mes] - intereses[mes]
        capital_vivo[mes] = capital_vivo[mes-1] - amortizacion[mes]
        capital_amortizado[mes] = capital_amortizado[mes-1] + amortizacion[mes]
        
    return pd.DataFrame(data={"mensualidades":mensualidades,"intereses":intereses, "amortizaciones":amortizacion,"capital_vivo":capital_vivo, "capital_amortizado":capital_amortizado, "a_anticipadas":a_anticipadas,"mensualidades_con_amort":mensualidades_con_amort}, index=np.arange(n_meses+1))

def calc_tae(prestamo, cuotas, gastos):
    flujos_caja = np.zeros(len(cuotas)+1) # +1 por el Mes 0
    flujos_caja[0] = -prestamo
    flujos_caja[1:] = cuotas + gastos
    TIR = npf.irr(flujos_caja)
    TAE = (1 + TIR)**12 - 1
    return {"TAE":TAE,"TIR":TIR}

In [ ]:
amortizaciones_dic = {1:5000,2:12000,3:12000,4:12000,5:10000,6:10000}
amortizaciones_futuras = crear_serie_amortizaciones(amortizaciones_dic,anios)
# euribor_cambios = {1:0.0005,2:0.0005,3:0.0005
#                    ,4:0.0005,5:0.0005,6:0.0005
#                    ,7:0.001,8:0.001,9:0.001,10:0.001,11:0.0005,12:0.0005,13:0.0005}
euribor_cambios = {1:0.001,2:0.001,3:0.001,4:0.001,5:0.001,6:0.001,7:0.001,8:0.001,9:0.001,10:0.001,11:0.001}
tin_variables = crear_serie_tins(euribor_inicial=-0.005,diferencial=0.0090,variaciones_euribor_dic=euribor_cambios,total_anios=30)
df = calc_series_mensuales_deudas_intereses_principal(tin_variables, anios,prestamo)
#df = calc_series_mensuales_deudas_intereses_principal(tin, anios,prestamo)

# Create figure with one Y Axe in this case
fig = go.Figure()
fig.add_trace(go.Scatter(x=df.index[1:], y=df.loc[:,"intereses"].iloc[1:],
                    mode='lines',
                    name='Interes pagado mensual'))
fig.add_trace(go.Scatter(x=df.index[1:], y=df.loc[:,"amortizaciones"].iloc[1:],
                    mode='lines',
                    name='Amortizacion mensual'))
fig.add_trace(go.Scatter(x=df.index[1:], y=df.loc[:,"mensualidades"].iloc[1:],
                    mode='lines',
                    name='Pago mensual'))
# Set x-axis title
fig.update_xaxes(title_text="Numero de mes")

# Set y-axes titles
fig.update_yaxes(title_text="Euros")

# Add figure title
fig.update_layout(
    title_text=f"Intereses vs principal con tin {100*tin}% - Total Intereses:{df.loc[:,'intereses'].iloc[1:].sum()}"
)

fig.show()

In [ ]:
# Create figure with one Y Axe in this case
fig = go.Figure()
fig.add_trace(go.Scatter(x=df.index[1:], y=df.loc[:,"capital_vivo"].iloc[1:],
                    mode='lines',
                    name='Capital Vivo restante'))
fig.add_trace(go.Scatter(x=df.index[1:], y=df.loc[:,"capital_amortizado"].iloc[1:],
                    mode='lines',
                    name='Capital amortizado total'))
fig.add_trace(go.Scatter(x=df.index[1:], y=np.ones(len(df.index[1:]))*prestamo,
                    mode='lines',
                    name='Prestamo Total'))
# Set x-axis title
fig.update_xaxes(title_text="Numero de mes")

# Set y-axes titles
fig.update_yaxes(title_text="Euros")

# Add figure title
fig.update_layout(
    title_text=f"Intereses vs principal con tin {100*tin}% - Total Intereses:{df.loc[:,'intereses'].iloc[1:].sum()}"
)

fig.show()

In [ ]:
# Total pagado segun cuando amortizas
q_amort = 15000
anios_amortizar = list(range(1,anios-3))
total_pagado = np.zeros(len(anios_amortizar))
for idx,anio in enumerate(anios_amortizar):
    amortizaciones_dic = {anio:q_amort,anio+1:q_amort,anio+2:q_amort}
    df = calc_series_mensuales_deudas_intereses_principal(tin, anios,prestamo,crear_serie_amortizaciones(amortizaciones_dic,anios))
    total_pagado[idx] = df.loc[:,"mensualidades_con_amort"].sum()

In [ ]:
# Create figure with one Y Axe in this case
fig = go.Figure()
fig.add_trace(go.Scatter(x=anios_amortizar, y=total_pagado,
                    mode='lines',
                    name='Total pagado al final del prestamo'))
# Set x-axis title
fig.update_xaxes(title_text=f"Anio amortizar {q_amort} Euros durante 3 anios")

# Set y-axes titles
fig.update_yaxes(title_text="Euros")

# Add figure title
fig.update_layout(
    title_text=f"Total pagado segun cuando se amorticen 3 x {q_amort} reduciendo cuotra, no años"
)

fig.show()

In [ ]:
# TIN vs Total pagado
tin_vec = np.arange(0.002,0.0185,0.0005)
total_pagado = np.zeros(len(tin_vec))
for tin_pos, tin_val in enumerate(tin_vec):
    total_pagado[tin_pos] = sum(calc_serie_cuota_mensual(tin=tin_val, n_meses=12*anios, prestamo=prestamo, as_array=True))

In [ ]:
# Create figure with one Y Axe in this case
fig = go.Figure()
fig.add_trace(go.Scatter(x=100*tin_a, y=total_pagado,
                    mode='lines',
                    name='Total pagado al final del prestamo'))
# Set x-axis title
fig.update_xaxes(title_text="Valor del TIN fijo en %")

# Set y-axes titles
fig.update_yaxes(title_text="Euros")

# Add figure title
fig.update_layout(
    title_text="TIN vs Total pagado con 30 anios"
)

fig.show()

In [ ]:
# Anios vs Total pagado
anios_a = np.arange(15,30,1)
total_pagado = np.zeros(len(anios_a))
for anios_pos, anios in enumerate(anios_a):
    total_pagado[anios_pos] = sum(calc_serie_cuota_mensual(tin=tin, n_meses=12*anios, prestamo=prestamo, as_array = True))

In [ ]:
# Create figure with one Y Axe in this case
fig = go.Figure()
fig.add_trace(go.Scatter(x=anios_a, y=total_pagado,
                    mode='lines',
                    name='Total pagado al final del prestamo'))
# Set x-axis title
fig.update_xaxes(title_text="Numero anios del prestamo")

# Set y-axes titles
fig.update_yaxes(title_text="Euros")

# Add figure title
fig.update_layout(
    title_text=f"Anios vs Total pagado con tin {100*tin}"
)

fig.show()

In [ ]:
# Anios vs Cuota mensual
anios_a = np.arange(15,30,1)
cuota_mensual = np.zeros(len(anios_a))
for anios_pos, anios in enumerate(anios_a):
    cuota_mensual[anios_pos] = calc_serie_cuota_mensual(tin=tin, n_meses=12*anios, prestamo=prestamo, as_array = False)

In [ ]:
# Create figure with one Y Axe in this case
fig = go.Figure()
fig.add_trace(go.Scatter(x=anios_a, y=cuota_mensual,
                    mode='lines',
                    name='Cuota Mensual'))
# Set x-axis title
fig.update_xaxes(title_text="Numero anios del prestamo")

# Set y-axes titles
fig.update_yaxes(title_text="Euros")

# Add figure title
fig.update_layout(
    title_text=f"Cuota Mensual vs Anios con tin {100*tin}"
)

fig.show()